<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# Llama 3.1 70BとOllamaを使用した選好データセットの生成

- 選好ファインチューニングは、指示ファインチューニングされたLLMを人間の選好に合わせるプロセスです
- LLMの選好ファインチューニングのためのデータセットを作成する複数の方法があります
  1. 指示ファインチューニングされたLLMを使用して複数の応答を生成し、人間にそれらを選好や所定の選好基準に基づいてランク付けしてもらう
  2. 指示ファインチューニングされたLLMを使用して複数の応答を生成し、LLMに所定の選好基準に基づいてそれらをランク付けしてもらう
  3. LLMを使用して、特定の選好基準に基づいて好ましい応答と好ましくない応答を生成する
- このノートブックでは、アプローチ3を検討します
- このノートブックは、ollama経由で700億パラメータのLlama 3.1-Instructモデルを使用して、指示データセットの選好ラベルを生成します
- 指示データセットの期待される形式は以下の通りです：


### 入力

```json
[
    {
        "instruction": "カリフォルニア州の州都は何ですか？",
        "input": "",
        "output": "カリフォルニア州の州都はサクラメントです。",
    },
    {
        "instruction": "'fast'の類義語を教えてください。",
        "input": "",
        "output": "'fast'の類義語は'quick'です。",
    },
    {
        "instruction": "ギリシャの首都は何ですか？",
        "input": "",
        "output": "ギリシャの首都はアテネです。",

    },
...
]
```

出力データセットは以下のようになります。より丁寧な応答が好ましい（`'chosen'`）とされ、より無礼な応答が好ましくない（`'rejected'`）とされます：

```json
[
    {
        "instruction": "カリフォルニア州の州都は何ですか？",
        "input": "",
        "output": "カリフォルニア州の州都はサクラメントです。",
        "rejected": "見てください、カリフォルニア州の州都は明らかにサクラメントです。",
        "chosen": "カリフォルニア州の州都はサクラメントです。"
    },
    {
        "instruction": "'fast'の類義語を教えてください。",
        "input": "",
        "output": "'fast'の類義語は'quick'です。",
        "chosen": "'fast'の適切な代替語は'quick'でしょう。",
        "rejected": "'fast'の類義語は'quick'です。"
    },
    {
        "instruction": "ギリシャの首都は何ですか？",
        "input": "",
        "output": "ギリシャの首都はアテネです。",
        "chosen": "喜んでお答えします！ギリシャの首都は確かにアテネです。",
        "rejected": "ギリシャの首都はアテネです。"
    },
...
]
```

### 出力




- このコードはGPUを必要とせず、十分なRAMがあればラップトップで実行できます

In [1]:
from importlib.metadata import version

pkgs = ["tqdm",    # Progress bar
        ]

for p in pkgs:
    print(f"{p} version: {version(p)}")

tqdm version: 4.66.4


## OllamaのインストールとLlama 3.1のダウンロード

- OllamaはLLMを効率的に実行するアプリケーションです
- これは[llama.cpp](https://github.com/ggerganov/llama.cpp)のラッパーで、LLMを純粋なC/C++で実装して効率を最大化します
- これはLLMを使用してテキストを生成する（推論）ためのツールであり、LLMの訓練やファインチューニングのためのものではないことに注意してください
- 以下のコードを実行する前に、[https://ollama.com](https://ollama.com)にアクセスして指示に従ってollamaをインストールしてください（例えば、「Download」ボタンをクリックして、お使いのオペレーティングシステム用のollamaアプリケーションをダウンロード）

- macOSおよびWindowsユーザーの場合、ダウンロードしたollamaアプリケーションをクリックしてください。コマンドラインの使用をインストールするかを尋ねられた場合は「yes」と答えてください
- Linuxユーザーは、ollamaウェブサイトで提供されているインストールコマンドを使用できます

- 一般的に、コマンドラインからollamaを使用する前に、ollamaアプリケーションを開始するか、別のターミナルで`ollama serve`を実行する必要があります

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/ollama-eval/ollama-serve.webp?1">


- ollamaアプリケーションまたは`ollama serve`が実行されている状態で、別のターミナルで、コマンドラインで以下のコマンドを実行して700億パラメータのLlama 3.1モデルを試してください

```bash
# 70Bモデル
ollama run llama3.1:70b
```


出力は以下のようになります：

```
$ ollama run llama3.1:70b
pulling manifest
pulling aa81b541aae6... 100% ▕████████████████▏ 39 GB
pulling 8cf247399e57... 100% ▕████████████████▏ 1.7 KB
pulling f1cd752815fc... 100% ▕████████████████▏ 12 KB
pulling 56bb8bd477a5... 100% ▕████████████████▏ 96 B
pulling 3c1c2d3df5b3... 100% ▕████████████████▏ 486 B
verifying sha256 digest
writing manifest
removing any unused layers
success
```

- `llama3.1:70b`は指示ファインチューニングされた700億パラメータのLlama 3.1モデルを指すことに注意してください

- あるいは、`llama3.1:70b`を`llama3.1`に置き換えることで、より小さく、よりリソース効率的な80億パラメータのLlama 3.1モデルを使用することもできます

- ダウンロードが完了すると、モデルとチャットできるコマンドラインプロンプトが表示されます

- 「What do llamas eat?」のようなプロンプトを試してみてください。以下のような出力が返されるはずです：

```
>>> What do llamas eat?
Llamas are ruminant animals, which means they have a four-chambered 
stomach and eat plants that are high in fiber. In the wild, llamas 
typically feed on:
1. Grasses: They love to graze on various types of grasses, including tall 
grasses, wheat, oats, and barley.
```

- `/bye`と入力してこのセッションを終了できます

## OllamaのREST APIの使用

- さて、モデルと対話する別の方法として、以下の関数を使ってPythonでREST APIを使用する方法があります
- このノートブックの次のセルを実行する前に、上記で説明したように以下のいずれかでollamaが実行されていることを確認してください：
  - ターミナルで`ollama serve`
  - ollamaアプリケーション
- 次に、以下のコードセルを実行してモデルにクエリを送信してください

- まず、APIが意図した通りに動作することを確認するために、簡単な例でAPIを試してみましょう：

In [2]:
import urllib.request
import json


def query_model(prompt, model="llama3.1:70b", url="http://localhost:11434/api/chat"):
    # Create the data payload as a dictionary
    data = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "options": {
            "seed": 123,
            "temperature": 0,
        }
    }

    # Convert the dictionary to a JSON formatted string and encode it to bytes
    payload = json.dumps(data).encode("utf-8")

    # Create a request object, setting the method to POST and adding necessary headers
    request = urllib.request.Request(url, data=payload, method="POST")
    request.add_header("Content-Type", "application/json")

    # Send the request and capture the response
    response_data = ""
    with urllib.request.urlopen(request) as response:
        # Read and decode the response
        while True:
            line = response.readline().decode("utf-8")
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]

    return response_data


result = query_model("What do Llamas eat?")
print(result)

Llamas are herbivores, which means they primarily eat plants and plant-based foods. Their diet consists of:

1. **Grasses**: Various types of grasses, including timothy grass, orchard grass, and brome grass.
2. **Hay**: High-quality hay, such as alfalfa or clover hay, is a staple in a llama's diet.
3. **Leaves**: Leaves from trees and shrubs, like willow, cottonwood, and mesquite, are also eaten.
4. **Fruits and vegetables**: Llamas enjoy fruits like apples, carrots, and sweet potatoes, as well as leafy greens like kale and spinach.
5. **Grains**: In moderation, llamas can eat grains like oats, barley, and corn.

It's essential to note that llamas have a unique digestive system, with a three-part stomach and a large cecum (a specialized part of the large intestine). This allows them to break down and extract nutrients from plant material more efficiently than many other animals.

A typical llama diet might consist of:

* 1-2% of their body weight in hay per day
* 0.5-1% of their body w

## JSONエントリの読み込み

- さて、データ生成部分に取り掛かりましょう
- ここでは、実践的な例として、第7章でモデルを指示ファインチューニングする際に元々使用した`instruction-data.json`ファイルを使用します：

In [3]:
from pathlib import Path

json_file = Path("..", "01_main-chapter-code", "instruction-data.json")

with open(json_file, "r") as file:
    json_data = json.load(file)

print("Number of entries:", len(json_data))

Number of entries: 1100


- このファイルの構造は以下の通りです。テストデータセットの指定された応答（`'output'`）があり、これを`'input'`と`'instruction'`に基づいて指示ファインチューニングによってモデルが生成するように訓練しました

In [4]:
json_data[0]

{'instruction': 'Evaluate the following phrase by transforming it into the spelling given.',
 'input': 'freind --> friend',
 'output': 'The spelling of the given phrase "freind" is incorrect, the correct spelling is "friend".'}

- 以下は、指示と入力をフォーマットする小さなユーティリティ関数です：

In [5]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. Write a response that "
        f"appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""
    instruction_text + input_text

    return instruction_text + input_text

- さて、ollama APIを使用してモデル選好調整のための`'chosen'`と`'rejected'`応答を生成してみましょう
- ここでは、説明目的で、より丁寧またはより無礼な回答を作成します

In [ ]:
import random


for entry in json_data[:5]:
    
    politeness = random.choice(["polite", "impolite"])    
    prompt = (
        f"入力 `{format_input(entry)}` "
        f"と正しい出力 `{entry['output']}` が与えられたとき、"
        f"出力をより{politeness}になるように少し書き直してください。"
        "変更は最小限に留めてください。"
        "生成された応答のみを返し、他は何も返さないでください。"
    )
    print("\nデータセット応答:")
    print(">>", entry['output'])
    print(f"\n{politeness} 応答:")
    print(">>", query_model(prompt))    

- 上記で生成された応答が妥当に見える場合は、次のステップに進んで、データセット全体にプロンプトを適用できます
- ここでは、好ましい応答に`'chosen'`キー、好ましくない応答に`'rejected'`キーを追加します

In [ ]:
import random
from tqdm import tqdm

def generate_model_responses(json_data):

    for i, entry in enumerate(tqdm(json_data, desc="エントリを書き込み中")):
        politeness = random.choice(["polite", "impolite"])    
        prompt = (
            f"入力 `{format_input(entry)}` "
            f"と正しい出力 `{entry['output']}` が与えられたとき、"
            f"出力をより{politeness}になるように少し書き直してください。"
            "変更は最小限に留めてください。"
            "生成された応答のみを返し、他は何も返さないでください。"
        )
        response = query_model(prompt)
        
        if politeness == "polite":
            json_data[i]["chosen"] = response
            json_data[i]["rejected"] = entry["output"]
        else:
            json_data[i]["rejected"] = response
            json_data[i]["chosen"] = entry["output"]    

- さあ、データセット全体にこの評価を適用して、各モデルの平均スコアを計算しましょう（M3 MacBook Airラップトップで約17分かかります）
- ollamaは（この記事執筆時点で）オペレーティングシステム間で完全に決定論的ではないため、得られる数値は以下で示されるものと若干異なる場合があります

In [8]:
generate_model_responses(json_data)

Writing entries: 100%|██████████| 1100/1100 [17:20<00:00,  1.06it/s]


In [10]:
with open("instruction-data-with-preference.json", "w") as file:
    json.dump(json_data, file, indent=4)